In [2]:
import pandas as pd
uriage_data = pd.read_csv("uriage.csv")
# uriage_data.describe()
# uriage_data.dtypes
uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])

# strtimefは日付タイプを文字列へ変換するメソッド
uriage_data["payment_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")
print(uriage_data.isnull().sum())
uriage_data.head()

purchase_date      0
item_name          0
item_price       387
customer_name      0
payment_month      0
dtype: int64


,purchase_date,item_name,item_price,customer_name,payment_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二,201907
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二,201904


In [13]:
pd.set_option('display.max_columns', 20)
uriage_data.groupby(["payment_month", "item_name"]).sum("item_price")

item_price
payment_month item_name            
201901        商品A            1500.0
              商品B            1600.0
              商品C            5400.0
              商品D            6000.0
              商品E            6500.0
...                             ...
201907        商品U           23100.0
              商品V           57200.0
              商品W           32200.0
              商品X           38400.0
              商品Y           27500.0

[176 rows x 1 columns]

In [4]:
pd.pivot_table(uriage_data, index="payment_month", columns="item_name", aggfunc="size", fill_value=0)
pd.pivot_table(uriage_data, index="payment_month", columns="item_name", values="item_price", aggfunc="sum", fill_value=0)

item_name,商品W,商 品 n,商品E,商品M,商品P,...,商品s,商品t,商品v,商品x,商品y
payment_month,,,,,,,,,,,
201901,0,1400,0,0,0,...,0,0,0,0,0
201902,0,0,0,0,0,...,1900,2000,2200,0,0
201903,0,0,500,1300,1600,...,0,0,0,0,0
201904,2300,0,0,0,0,...,1900,0,0,0,0
201905,0,0,0,0,0,...,0,0,0,0,2500
201906,0,0,0,0,0,...,0,0,0,2400,0
201907,0,0,0,0,0,...,0,0,0,0,0


In [5]:
# データの揺れを補正する
print(len(uriage_data["item_name"].unique()))
uriage_data["item_name"] = uriage_data["item_name"].str.upper()
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")
uriage_data["item_name"] = uriage_data["item_name"].str.replace("　", "")

print(len(uriage_data["item_name"].unique()))
t_list = uriage_data["item_name"].unique()
t_list.sort()
t_list

99
26


array(['商品A', '商品B', '商品C', '商品D', '商品E', '商品F', '商品G', '商品H', '商品I',
       '商品J', '商品K', '商品L', '商品M', '商品N', '商品O', '商品P', '商品Q', '商品R',
       '商品S', '商品T', '商品U', '商品V', '商品W', '商品X', '商品Y', '商品Z'],
      dtype=object)

In [24]:
uriage_data.head(20)

,purchase_date,item_name,item_price,customer_name,payment_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904
5,2019-03-20 19:16:01,商品S,1900.0,堀江佑,201903
6,2019-05-18 19:16:53,商品A,100.0,深井照生,201905
7,2019-04-18 00:14:21,商品V,2200.0,牧田玲那,201904
8,2019-01-10 15:51:01,商品O,1500.0,堀北雅彦,201901
9,2019-01-28 10:47:03,商品A,100.0,大地礼子,201901


In [26]:
uriage_data.isnull().any(axis=0)

flg_is_null = uriage_data["item_price"].isnull()
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"] == trg), "item_price"].max()
    uriage_data["item_price"].loc[(flg_is_null) & (uriage_data["item_name"] == trg)] = price

In [28]:
uriage_data.isnull().any(axis=0)
uriage_data.head(5)

,purchase_date,item_name,item_price,customer_name,payment_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


In [36]:
kokyaku_data = pd.read_excel("kokyaku_daicho.xlsx")
kokyaku_data["顧客名"].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

In [37]:
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace(" ", "")
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace("　", "")
kokyaku_data["顧客名"].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

In [40]:
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


In [43]:
flg_is_serial = kokyaku_data["登録日"].astype("str").str.isdigit()
flg_is_serial.sum()

22

In [44]:
pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "登録日"].astype("float"), unit="D")

1     42782 days
3     42872 days
4     43127 days
21    42920 days
27    42901 days
47    42741 days
49    42929 days
53    42833 days
76    43188 days
80    43110 days
99    42885 days
114   43254 days
118   43129 days
122   43206 days
139   42880 days
143   42818 days
155   42754 days
172   43181 days
179   42743 days
183   42940 days
186   43294 days
192   43259 days
Name: 登録日, dtype: timedelta64[ns]

In [46]:
from_serial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "登録日"].astype("float"), unit="D") + pd.to_datetime("1900/01/01")
from_serial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 登録日, dtype: datetime64[ns]

In [49]:
from_string = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, "登録日"])
from_string

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [50]:
kokyaku_data["登録日"] = pd.concat([from_serial, from_string])
kokyaku_data

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


In [52]:
kokyaku_data["登録日"].astype("str").str.isdigit().sum()

0

In [54]:
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29


In [55]:
uriage_data.head()

,purchase_date,item_name,item_price,customer_name,payment_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


In [60]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on="customer_name", right_on="顧客名", how="left")
join_data.head()

,purchase_date,item_name,item_price,customer_name,payment_month,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07


In [61]:
join_data = join_data.drop("customer_name", axis=1)
join_data.head()

,purchase_date,item_name,item_price,payment_month,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,商品A,100.0,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,商品S,1900.0,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,商品A,100.0,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,商品Z,2600.0,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,商品A,100.0,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07


,purchase_date,item_name,item_price,customer_name,payment_month,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07
